**Import Libraries**

In [1]:
import pandas as pd

In [2]:
pd.set_option('display.max_columns', None)

**Function to clean house sale data**

In [3]:
def salehouse(x):

    #change titles to lower case 
    lower = []
    for i in x.columns:
        if ' ' in i:
            i = i.replace(' ', '_')
            lower.append(i.lower())
        else:    
            lower.append(i.lower())
    x.columns = lower


    #Drop unneccessary columns
    x = x.drop(columns=['title','unnamed:_0','raw_address','is_exact_address_hidden','sq_mt_useful','is_furnished','is_kitchen_equipped','sq_mt_allotment','n_floors','is_accessible','has_green_zones','energy_certificate','has_parking','has_private_parking','has_public_parking','is_parking_included_in_price','parking_price','is_orientation_north','is_orientation_west','is_orientation_south','is_orientation_east','has_fitted_wardrobes','are_pets_allowed','operation','rent_price_by_area','is_rent_price_known','buy_price_by_area','door','portal'])

    
    #Split subtitle for the neighbourhood, rename subtitle to Neighbourhood
    x['subtitle'] = x['subtitle'].str.split(",", n = 0, expand = True)[0]
    x = x.rename(columns={'subtitle':'neighbourhood','has_central_heating':'central_heating','has_individual_heating':'individual_heating','has_ac':'aircon','has_lift':'lift','is_exterior':'exterior','has_garden':'garden','has_pool':'pool','has_terrace':'terrace','has_balcony':'balcony','has_storage_room':'storage_room'})
    
    
    #split the neighbourhood id to have the district name and price per m2
    x['neighborhood_id'] = x['neighborhood_id'].str.split("(", n = 0, expand = True)[1]
    x['sq_m_price_€/m2'] = x['neighborhood_id'].str.split("-", n = 0, expand = True)[0]
    x['sq_m_price_€/m2'] = x['sq_m_price_€/m2'].str.split(" €", n = 0, expand = True)[0]
    x['district'] = x['neighborhood_id'].str.split(": ", n = 1, expand = True)[1]
    x = x.drop(columns=['neighborhood_id'])
    
    
    #calculate own rent price, due to problems in the data
    x['rent_price'] = round(x['buy_price']/122.6)
    x['rent_price'] = x['rent_price'].astype(int)
    
    
    #drop nan values
    x = x.dropna(subset=['street_name'])
    
    
    #deal with NaN values
    x['n_bathrooms'] = x['n_bathrooms'].fillna(x['n_bathrooms'].mean()) 
    x['street_number'].fillna('0')
    x['is_floor_under'].fillna('False')
    
    
    #Extract the house types from house_type_id_column and remove any spaces
    x['house_type_id'] = x['house_type_id'].str.split(":", n = 0, expand = True)[1]
    x['house_type_id'] = x['house_type_id'].str.strip()
    
    
    # Fill NaN values to '9999' and change built_year to int
    x['built_year'] = x['built_year'].fillna('9999')
    x['built_year'] = x['built_year'].astype(int)


    # DEAL WITH NAN VALUES

    #zeros
    x['sq_mt_built'] = x['sq_mt_built'].fillna(0)
    x['street_number'] = x['street_number'].fillna(0)
    x['floor'] = x['floor'].fillna(0)


    #Others
    x['house_type_id'] = x['house_type_id'].fillna('Desconocido')
    
    
    #map 0 and 1
    def maps(x):
        x = x.fillna(0)
        x = x.map({True : 1, 0:0})
        return x
    
    
    #Deal with boolean values
    x['is_floor_under'] = maps(x['is_floor_under'])
    x['is_new_development'] = maps(x['is_new_development'])
    x['central_heating'] = maps(x['central_heating'])
    x['individual_heating'] = maps(x['individual_heating'])
    x['aircon'] = maps(x['aircon'])
    x['lift'] = maps(x['lift'])
    x['exterior'] = maps(x['exterior'])
    x['garden'] = maps(x['garden'])
    x['pool'] = maps(x['pool'])
    x['terrace'] = maps(x['terrace'])
    x['balcony'] = maps(x['balcony'])
    x['storage_room'] = maps(x['storage_room'])
    
    
    #Create one 'heating column'
    x['heating'] = x['central_heating'] + x['individual_heating']
    
    
    #Deal with data conversion problem
    x['sq_m_price_€/m2'] = x['sq_m_price_€/m2'].str.strip()

    exc = []
    for i in x['sq_m_price_€/m2']:
        if 'None' in i:
            exc.append(0)
        else:
            exc.append(i)
    x['sq_m_price_€/m2'] = exc
    
    x['sq_m_price_€/m2'] = x['sq_m_price_€/m2'].astype(float)
    
    
    #Map floor values to numeric
    x['floor'] = x['floor'].map({'3':3, '4':4, '1':1, 'Bajo':0, '7':7, '2':2, 'Semi-sótano': -1, '5':5, 0:0, '6':6, 'Entreplanta exterior':1, '8':8, 'Entreplanta interior':1, 'Entreplanta':1,'9':9, 'Semi-sótano exterior': -1, 'Sótano interior': -1,'Semi-sótano interior': -1, 'Sótano' : -1, 'Sótano exterior' : -1})
    x['floor'] = x['floor'].astype(int)
    
    
    #Assign listing type for future filtering
    x['type'] = 'sale'
    
    
    #Determine columns to be in final dataframe
    x = x[['id','district','neighbourhood','latitude','longitude','house_type_id','street_name','street_number','n_rooms','n_bathrooms','floor','sq_mt_built','built_year','heating','aircon','lift','exterior','garden','pool','terrace','balcony','storage_room','rent_price','buy_price','sq_m_price_€/m2','type']]
   

    ##DEALING WITH PROBLEM VALUES IN NEIGHBOURHOOD/DISTRICT WITH OTHER DFS##
    
    #Neighbourhood renaming values
    neighbs = []
    for i in x['neighbourhood']:
        if 'Barajas' in i:
            neighbs.append('Aeropuerto')
        elif 'Campo de las Naciones-Corralejos' in i:
            neighbs.append('Corralejos')        
        elif 'Chueca-Justicia' in i:
            neighbs.append('Justicia') 
        elif 'Huertas-Cortes' in i:
            neighbs.append('Cortes') 
        elif 'Lavapiés-Embajadores' in i:
            neighbs.append('Embajadores')
        elif 'Malasaña-Universidad' in i:
            neighbs.append('Universidad')
        elif 'Bernabéu-Hispanoamérica' in i:
            neighbs.append('Hispanoamérica')
        elif 'Nuevos Ministerios-Ríos Rosas' in i:
            neighbs.append('Rios Rosas')
        elif 'Tres Olivos - Valverde' in i:
            neighbs.append('Valverde')
        elif 'Conde Orgaz-Piovera' in i:
            neighbs.append('Piovera')     
        elif 'Valdebebas - Valdefuentes' in i:
            neighbs.append('Valdefuentes')
        elif 'Virgen del Cortijo - Manoteras' in i:
            neighbs.append('Manoteras')
        elif 'Águilas' in i:
            neighbs.append('Aguilas')
        elif 'Los Cármenes' in i:
            neighbs.append('Cármenes')
        elif 'Barrio de Salamanca' in i:
            neighbs.append('Salamanca')
        elif 'Cuzco-Castillejos' in i:
            neighbs.append('Castillejos')
        elif 'Ventilla-Almenara' in i:
            neighbs.append('Almenara')
        elif '12 de Octubre-Orcasur' in i:
            neighbs.append('Orcasur')
        elif 'El Cañaveral - Los Berrocales' in i:
            neighbs.append('El Cañaveral')
        elif 'Valdebernardo - Valderribas' in i:
            neighbs.append('Valdebernardo')    
        elif 'Ensanche de Vallecas' in i:
            neighbs.append('Valdecarros')
        elif 'Los Ángeles' in i:
            neighbs.append('Los Angeles')
        elif 'San Cristóbal' in i:
            neighbs.append('San Cristobal')
        else:
            neighbs.append(i)
    x['neighbourhood'] = neighbs    


    #District renaming values
    dist = []
    for i in x['district']:
        if 'Fuencarral' in i:
            dist.append('Fuencarral - El Pardo')
        elif 'Moncloa' in i:
            dist.append('Moncloa - Aravaca')
        else:
            dist.append(i)
    x['district'] = dist

    #strip neighbourhood & district to remove any spaces
    x['neighbourhood'] = x['neighbourhood'].str.strip()
    x['district'] = x['district'].str.strip()

    
    return x


**Import csv, Dataframe Before cleaning...**

In [4]:
sales = pd.read_csv('raw_data/Madrid Housing Market/houses_Madrid.csv')
sales.head(2)

,Unnamed: 0,id,title,subtitle,sq_mt_built,sq_mt_useful,n_rooms,n_bathrooms,n_floors,sq_mt_allotment,latitude,longitude,raw_address,is_exact_address_hidden,street_name,street_number,portal,floor,is_floor_under,door,neighborhood_id,operation,rent_price,rent_price_by_area,is_rent_price_known,buy_price,buy_price_by_area,is_buy_price_known,house_type_id,is_renewal_needed,is_new_development,built_year,has_central_heating,has_individual_heating,are_pets_allowed,has_ac,has_fitted_wardrobes,has_lift,is_exterior,has_garden,has_pool,has_terrace,has_balcony,has_storage_room,is_furnished,is_kitchen_equipped,is_accessible,has_green_zones,energy_certificate,has_parking,has_private_parking,has_public_parking,is_parking_included_in_price,parking_price,is_orientation_north,is_orientation_west,is_orientation_south,is_orientation_east
0,0,21742,"Piso en venta en calle de Godella, 64","San Cristóbal, Madrid",64.0,60.0,2,1.0,NaN,NaN,NaN,NaN,"Calle de Godella, 64",False,Calle de Godella,64,NaN,3,False,NaN,Neighborhood 135: San Cristóbal (1308.89 €/m2)...,sale,471,NaN,False,85000,1328,True,HouseType 1: Pisos,False,False,1960.0,NaN,NaN,NaN,True,NaN,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D,False,NaN,NaN,NaN,NaN,False,True,False,False
1,1,21741,Piso en venta en calle de la del Manojo de Rosas,"Los Ángeles, Madrid",70.0,NaN,3,1.0,NaN,NaN,NaN,NaN,Calle de la del Manojo de Rosas,True,Calle de la del Manojo de Rosas,NaN,NaN,4,False,NaN,Neighborhood 132: Los Ángeles (1796.68 €/m2) -...,sale,666,NaN,False,129900,1856,True,HouseType 1: Pisos,True,False,NaN,NaN,NaN,NaN,NaN,True,True,True,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,en trámite,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**...and After**

In [5]:
sale = salehouse(sales)
sale.head(2)

,id,district,neighbourhood,latitude,longitude,house_type_id,street_name,street_number,n_rooms,n_bathrooms,floor,sq_mt_built,built_year,heating,aircon,lift,exterior,garden,pool,terrace,balcony,storage_room,rent_price,buy_price,sq_m_price_€/m2,type
0,21742,Villaverde,San Cristobal,NaN,NaN,Pisos,Calle de Godella,64,2,1.0,3,64.0,1960,0,1,0,1,0,0,0,0,0,693,85000,1308.89,sale
1,21741,Villaverde,Los Angeles,NaN,NaN,Pisos,Calle de la del Manojo de Rosas,0,3,1.0,4,70.0,9999,0,0,1,1,0,0,1,0,0,1060,129900,1796.68,sale


**Merge dfs to get latitude and longitude information**

In [6]:
latlong = pd.read_csv('cleaned_data/latlong.csv')
latlong.head()

,district,neighbourhood,latitude,longitude
0,Arganzuela,Acacias,40.40097,-3.70109
1,Arganzuela,Atocha,40.40152,-3.68443
2,Arganzuela,Chopera,40.39445,-3.69626
3,Arganzuela,Delicias,40.39848,-3.69692
4,Arganzuela,Imperial,40.41327,-3.71866


In [7]:
MHS_Merged = pd.merge(sale,latlong, how='inner', left_on='neighbourhood', right_on='neighbourhood')
MHS_Merged = MHS_Merged.rename(columns={'district_x':'district','latitude_y':'latitude','longitude_y':'longitude'})
MHS_Merged = MHS_Merged[['id','district','neighbourhood','latitude','longitude','house_type_id','street_name','street_number','n_rooms','n_bathrooms','floor','sq_mt_built','built_year','heating','aircon','lift','exterior','garden','pool','terrace','balcony','storage_room','rent_price','buy_price','sq_m_price_€/m2','type']]
MHS_Merged.head(2)

,id,district,neighbourhood,latitude,longitude,house_type_id,street_name,street_number,n_rooms,n_bathrooms,floor,sq_mt_built,built_year,heating,aircon,lift,exterior,garden,pool,terrace,balcony,storage_room,rent_price,buy_price,sq_m_price_€/m2,type
0,21742,Villaverde,San Cristobal,40.34326,-3.6875,Pisos,Calle de Godella,64,2,1.0,3,64.0,1960,0,1,0,1,0,0,0,0,0,693,85000,1308.89,sale
1,21728,Villaverde,San Cristobal,40.34326,-3.6875,Pisos,Calle GODELLA,0,3,1.0,3,64.0,1960,1,0,0,1,0,0,1,0,0,587,72000,1308.89,sale


**Export Final Df to CSV**

In [8]:
#MHS_Merged.to_csv('Madrid_Sales.csv', index = False)